# Literate computing should be permissive.

When building an idea, natural language typically proceeds a program, respectively.  Currently, notebook composition decouples natural language and code to Markdown and Code cells, respectively.  In [a previous document](http://nbviewer.jupyter.org/github/deathbeds/deathbeds.github.io/blob/master/deathbeds/2018-09-02-SyntaxError-fallback.ipynb) we outlined the differences and similarities between the two cell types.

A literate document desires a tight connection between natural language and code.  As authors, we are still in the
early stages of learning to write about code in common conversations. 

In this notebook, we try to mitigate `SyntaxError` by allowing prose in code cells.  The code cells, if they are not valid source code, defer to a Markdown transformer that executes the inline and block code in the Markdown.  

A computer may not execute code in a Markdown cell, but a consumer may replicate or reuse that code.  __Anything that is code should compute__.  Authoring markdown in code cells allows an author to validate the code in their prose.

> A benefit of composing literate particles in code cells is that an author may use tab completion and inspection.  I've found that this feature promotes better weaving of narrative and code.

In [1]:
    from deathbeds.__Markdown_code_cells import CallableTransformer, CodeRenderer
    from textwrap import dedent; import fnmatch, nbconvert; from nbformat import v4
    import IPython, sys, traceback, warnings, black, itertools
    from contextlib import contextmanager
    from toolz.curried import *
    from deathbeds.__Custom_display_formatting import triggers
    ip = get_ipython()

The first errors that could be encountered in `IPython` when interacting with code are `SyntaxError` & 
`IndentationError`; `assert issubclass(IndentationError, SyntaxError)`.  These `Exceptions` deny any code from being compiled and executing.  

There are a few `SyntaxError` `Exception`s

In [2]:
[x for x in vars(__import__('builtins')).values() if isinstance(x, type) and issubclass(x, SyntaxError)]

[SyntaxError, IndentationError, TabError]

In our problem, we have to create some `Exception`s that are `SyntaxError`s.  [They must base class `SyntaxError` because we will using the
`ip.input_transformer_manager`](https://ipython.readthedocs.io/en/stable/config/inputtransforms.html) & any 
other `Exception` will unregister the transformer.  

A special `SyntaxError` case to consider is when a cell is pure Markdown containing no code elements.  In this situation,
we display a `NoMarkdownCodeWarning` `UserWarning` before displays the `SyntaxError`.

In [3]:
    class NoMarkdownCodeException(SyntaxError): ...
    class NoMarkdownCodeWarning(UserWarning): ...

## The Input Transformer.

Our `Literate` renderer will extract inline and block code as code input.

In [13]:
    class Literate(CallableTransformer, CodeRenderer):
        def code(self, node, entering): self.nodes.append(node)

        def __call__(self, str):
            self.nodes = []
            
            IPython.display.display(IPython.display.Markdown(str))
            
            current, lines, str = 0, str.splitlines(), CodeRenderer.__call__(self, str)
            
            if lines and not(self.nodes or str.strip()):
                warnings.warn(
                    NoMarkdownCodeWarning("Cannot interpret Markdown cells with no block or inline code objects."),
                    NoMarkdownCodeWarning, stacklevel=1000)

                raise NoMarkdownCodeException("No code objects discoved in the Markdown code.") from Exception
            
            
            str = dedent(str or '\n').splitlines()
            
            while """Increment the lines until there are enough of them.
            """ and (len(str) < len(lines)): str.append("")

            for node in self.nodes:
                for current in range(current, len(lines)):
                    try:
                        id = lines[current].index(f"`{node.literal}`")
                        break
                    except ValueError: ...
                str[current] += ('; ' if str[current].strip() else '') + node.literal
                
            try:
                format = black.format_str('\n'.join(str), 60)
                show = False
                for line in format.splitlines():
                    show = show or bool(line.strip())
                    show and ip.auto_rewrite_input(line)
            except: ...
            return '\n'.join(str)

`swap_input_transformer` is a `contextmanager` that temporaily attaches `Literate` to the transformers
on the 🥇 `SyntaxError` using the `ip.input_transformer_manager.physical_line_transforms`.

In [15]:
@contextmanager
def swap_input_transformer(object, position=0):
    ip = get_ipython()
    yield ip.input_transformer_manager.physical_line_transforms.insert(position, object) or ip
    ip.input_transformer_manager.physical_line_transforms = [
        x for x in get_ipython().input_transformer_manager.physical_line_transforms
        if not isinstance(x, type(object))]

## Including other syntaxes

[`import deathbeds.__Custom_display_formatting`](2018-09-03-Custom-display-formatting.ipynb) supports more syntaxes than Markdown.  We will include syntaxes for displaying images, webpages, and graphviz in a document.  If any 
of the values in `triggers` is found then we use Custom Display Formatters instead.  _These custom displays must be wrapped in strings when used in indented code blocks._

`istrigger` determines if the input string is valid trigger for the Custom display formatters for 
__graphviz__, links, and flexbox layouts..

In [16]:
    def istrigger(str=None): return list(f(str or ip.user_ns['In'][-1]) for f in triggers)

`fallback` is the custom `IPython` exception handler.  

1. It checks if the string is a trigger, if it is then the source is converted to a string.
2. If these conditions fail then we defer to default mode.

In [17]:
    def fallback(ip, type, Exception, tb, **kwargs): 
        with modify_custom_exceptions(SyntaxError):
            code = ip.user_ns['In'][-1]
            if any(istrigger(code)):
                quote = '"""'
                if quote in code: quote = "'''"
                ip.run_cell(quote+code+quote)
            else: 
                with swap_input_transformer(Literate()): 
                    ip.run_cell(code)

Since `fallback` uses the `ip.run_cell` we must make sure we don't get stuck in a `RecursionError`.  `modify_custom_exceptions`
is a `contextmanager` to swap out the current `Exception` that is being treated in a special way.

In [18]:
    @contextmanager
    def modify_custom_exceptions(Exception):
        custom_exceptions = groupby(lambda x: issubclass(x, SyntaxError), ip.custom_exceptions)
        ip.custom_exceptions = tuple(x for x in custom_exceptions.get(False, []))
        yield
        ip.custom_exceptions = tuple(sum(custom_exceptions.values(), list()))

### Create the extensions.

`load_ipython_extension` allows this extension to be used with <code>%load_ext</code>, and adding `unload_ipython_extension`
allows it to be used with <code>%reload_ext</code>.

In [19]:
    def load_ipython_extension(ip): 
        ip.set_custom_exc((SyntaxError,), fallback)
        %reload_ext deathbeds.__Custom_display_formatting
    def unload_ipython_extension(ip): 
        %unload_ext deathbeds.__Custom_display_formatting
        ip.custom_exceptions = tuple(set(ip.custom_exceptions)-{SyntaxError})

# Discussion

This notebook was created using the `SyntaxError` fallback, this tool made it easier to weave the narrative together with
code because of tab completion.  Also, it prevented me from placing misspelled or invalid code in the Markdown cells.

### tests

The tests should test different inputs and assure errors are being handled predictably.

In [20]:
    def _load_extension_text():
        %load_ext deathbeds.__SyntaxError_fallback
        %unload_ext deathbeds.__SyntaxError_fallback
    def _handle_valid_code(): ...
    def _handle_invalid_code(): ...
    def _handle_string_transformers(): ...
    def _handle_markdown(): ...
    def _handle_markdown_with_syntaxerror(): ...
    def _handle_markdown_with_other_errors(): ...
    if __name__ == '__main__':
        _load_extension_text()
        load_ipython_extension(get_ipython())
